<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
    
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 

<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.




<p></p><p></p><p></p>





**Le problème posé :**


*Stack Overflow est un site célèbre de question-réponses liées au développement informatique. Pour poser une question sur ce site, il faut entrer plusieurs tags de manière à retrouver facilement la question par la suite. Pour les utilisateurs expérimentés cela ne pose pas de problème, mais pour les nouveaux utilisateurs, il serait judicieux de suggérer quelques tags relatifs à la question posée.*

*Amateur de Stack Overflow, qui vous a souvent sauvé la mise, vous décidez d'aider la communauté en retour. Pour cela, vous développez un système de suggestion de tags pour le site. Celui-ci prendra la forme d’un algorithme de machine learning qui assigne automatiquement plusieurs tags pertinents à une question.*



**Solutions mises en oeuvre**

Les solutions de suggestion de tags présentés ici se basent sur des modèles *non-supervisés* de machine learning.

Les modèles mis en oeuvre : 
    * Clusterisation par K-means
    * LDA
    * Word2Vec

Ce notebook utilise les données issues du notebook **P6_DadaAnalysis.ipynb**



# 3. Unsupervised methods

TAGS are regarded as keywords.

Aims of this section is to create data points then identify clusters from this set of data.

These data-points are digital representation of the cleaned documents dataset.

For each-one of the clusters, most representative terms will be identified.

These terms will be considered as TAGs.

Differents embeddings techniques will be used in order to create data-points : 

* Bag of words
* Co-occurence
* TF-IDF

**Loading train dataset and extract users questions from Body column**

In [1]:
import pandas as pd
file_name="./data/train_StackOverFlow_BodyTitleTags.csv"
df_sof_train=pd.read_csv(file_name)
print(df_sof_train.shape)

(24604, 5)


In [ ]:
type(df_sof_train['Body'])

**Standardize `Body` column from dataset**

In [ ]:
import p6_util
ser_document_std = p6_util.p6_df_standardization(df_sof_train['Body'],is_stemming=False, is_lem=False)

**Save dataframe with standardized `Body` column**

In [ ]:
import p5_util
file_name="./data/df_document_std.dump"
p5_util.object_dump(ser_document_std, file_name)

**Load standardized corpus**

In [ ]:
import p5_util
file_name="./data/df_document_std.dump"
ser_document_std = p5_util.object_load(file_name)
print(ser_document_std.shape)

In [ ]:
ser_document_std = ser_document_std[:16359]
print(ser_document_std.shape)

#### Corpus vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# When building vacabulary, terms with frequency document < p_min_df are ignored.
p_min_df = 0.001
# When building vacabulary, terms with frequency document> p_max_df are ignored.
p_max_df = 1.


dict_sof_document = ser_document_std.to_dict()

ngram1=2
ngram2=2
vectorizer=TfidfVectorizer(norm="l2", use_idf=True, min_df=p_min_df, max_df=p_max_df, ngram_range=(ngram1, ngram2))

csr_matrix = vectorizer.fit_transform(dict_sof_document.values())

print(csr_matrix.shape)

**Save vectorization with ngram (2,2) mode**

In [ ]:
import p5_util
file_name="./data/csr_matrix_tdif_ngram_2_2.dump"
p5_util.object_dump(csr_matrix, file_name)

file_name="./data/vectorizer_tdif_ngram_2_2.dump"
p5_util.object_dump(vectorizer, file_name)

## 3.1 Kmeans clustering

**Load vectorization operator and CSR matrix**

In [ ]:
import p5_util
file_name="./data/csr_matrix_tdif_ngram_2_2.dump"
csr_matrix = p5_util.object_load(file_name)
print(csr_matrix.shape)

file_name="./data/vectorizer_tdif_ngram_2_2.dump"
vectorizer = p5_util.object_load(file_name)

**Proceed to Kmeans clustering**

In [ ]:
from sklearn.cluster import KMeans
n_cluster=100
cluster_kmean=KMeans(n_clusters=n_cluster).fit(csr_matrix)

print(len(cluster_kmean.labels_))

**Save Kmeans clustering**

In [ ]:
file_name="./data/cluster_kmeans_labels_100.dump"
p5_util.object_dump(cluster_kmean.labels_, file_name)

**For each cluster build a TAG list**

In [ ]:
file_name="./data/cluster_kmeans_labels_100.dump"
cluster_kmean_labels = p5_util.object_load(file_name)

In [ ]:
import p6_util
p_tag_ratio=1.0
dict_list_cluster_tag, dict_cluster_stat, dict_df_freq_cluster_tag \
= p6_util.get_dict_list_cluster_tag(cluster_kmean_labels, dict_sof_document, vectorizer, p_tag_ratio)

In [ ]:
import p5_util
file_name="./data/dict_list_cluster_tag.dump"
p5_util.object_dump(dict_list_cluster_tag, file_name)

#### Plot cluster distribution

In [ ]:
import pandas as pd
# Convert dictionary as dataframe allowing having statisticals values
df_cluster_stat = pd.DataFrame.from_dict(dict_cluster_stat, orient='index')
df_cluster_stat.rename(columns={0:'cluster'},inplace=True)

min_cluster_count = min(df_cluster_stat.cluster)
max_cluster_count = max(df_cluster_stat.cluster)

#Get outliers clusters : mix and max values
cluster_max_id = df_cluster_stat.query("cluster == "+str(max_cluster_count), inplace=False).index[0]
cluster_min_id = df_cluster_stat.query("cluster == "+str(min_cluster_count), inplace=False).index[0]

print("Cluster {} : elements= {} ".format(cluster_min_id, min_cluster_count))
print("Cluster {} : elements= {} ".format(cluster_max_id, max_cluster_count))


**Display Questions distribution among clusters**

In [ ]:
df_cluster_stat.query("cluster <= 1000", inplace=True)
ax = df_cluster_stat.plot.bar(figsize=(7, 7), title="Questions distribution", color='orange', fontsize=14)

**Display cluster with lower number of elements**

In [ ]:
import p6_util_plot
df_freq_cluster_tag = dict_df_freq_cluster_tag[cluster_min_id]
dict_freq_cluster_tag = df_freq_cluster_tag.to_dict()['Freq']
p6_util_plot.display_word_cloud(dict_freq_cluster_tag)

In [ ]:
print(dict_list_cluster_tag[cluster_min_id])
print("\n")
#print(dict_list_cluster_tag[cluster_max_id])

**Display cluster with greater number of elements**

In [ ]:
import p6_util_plot
df_freq_cluster_tag = dict_df_freq_cluster_tag[cluster_max_id]
dict_freq_cluster_tag = df_freq_cluster_tag.to_dict()['Freq']
p6_util_plot.display_word_cloud(dict_freq_cluster_tag)

## Testing process

Questions (Posts) are vectorized.

Vectorization leads to a Matrix.

Transpozed matrix is clusterized : transpozed matrix columns are questions (Posts).

A clustering algorythm is applied over transposed matrix. Each cluster compound a set of questions.

Each cluster is assigned with some TAGs that way : 

* In any cluster, TAGs are picked up from vectors components.

* For each one of the cluster element : 
    * Greatest components values are picked up.
    * TAGs assigned are vocabulary terms matching with picked-up values.


**Extract a question from test data-set**

**Loading test dataset**

In [ ]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
df_sof_test=pd.read_csv(file_name)

In [ ]:
import random
import p6_util

i_random = random.choice(range(0, df_sof_test.shape[0]))
question= df_sof_test.Body.iloc[i_random]
print("\nQuestion : \n")
print(question)

**Question is standardized**

In [ ]:
import p6_util

df_std =p6_util.preprocess_post(question, is_stemming=False, is_lem=True, is_stopword=True\
    ,is_stopverb=True, is_stopalfanum=True)

In [ ]:
post_std = df_std.to_dict()['Body'][0]
print(post_std)

**Convert standardized post into bigram terms**

In [ ]:
import nltk

post_std_bigrams = nltk.bigrams(post_std.split(' '))


print(post_std)
list_key_words = [ tuple_bigram[0]+'_'+tuple_bigram[1] for tuple_bigram in post_std_bigrams]
list_key_words

**Search for cluster question belongs to**

In [ ]:
import p5_util
file_name="./data/dict_list_cluster_tag.dump"
dict_list_cluster_tag = p5_util.object_load(file_name)

In [ ]:
import p6_util
dict_cluster_freq = p6_util.get_tag_intersect_cluster_list_tag(post_std, dict_list_cluster_tag)

In [ ]:
df_cluster_freq = pd.DataFrame.from_dict(dict_cluster_freq, orient='index')

df_cluster_freq.rename(columns={0:'Frequency'}, inplace=True)
df_cluster_freq.sort_values(by=['Frequency'],  ascending=False, inplace=True)
print("Question has been attached to cluster = "+str(df_cluster_freq.iloc[0]))
dict_cluster_freq

## 3.2. LDA

#### Load TDIF Matrix and related vectorizer

In [ ]:
import p5_util
file_name="./data/csr_matrix_tdif.dump"
csr_matrix = p5_util.object_load(file_name)

print(csr_matrix.shape)

In [ ]:
import p5_util
file_name="./data/vectorizer_tdif_ngram_2_2.dump"
vectorizer = p5_util.object_load(file_name)

In [ ]:
tfidf_feature_names = vectorizer.get_feature_names()

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(lda.components_):
        message = "Topic %d: " % (topic_idx)
        message += " / ".join([feature_names[i] \
                               for i in topic.argsort()[:-no_top_words - 1:-1] \
                               if i<len(feature_names)])
        print(message)

**LDA with full standardized dataset**

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
no_topics = 100

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5\
                                , learning_method='online', learning_offset=50.,random_state=0).fit(csr_matrix)

In [ ]:
len(tfidf_feature_names)

In [ ]:
lda.components_.shape

In [ ]:
no_top_words = 1
display_topics(lda, tfidf_feature_names, no_top_words)

In [ ]:
from sklearn.cluster import DBSCAN
p_eps=0.1
dbscan = DBSCAN(eps=p_eps,min_samples=350, n_jobs=3)

if 'cluster' in df_sof.columns:
    del(df_sof['cluster'])
pred_dbscan =  dbscan.fit_predict(df_customers)

import numpy as np
np.unique(pred_dbscan)

**DBSCAN custering**

CSR matrix is converted into dataframe in order to feed cluster algorithm

In [ ]:
import pandas as pd
df_sof = pd.DataFrame(csr_matrix.toarray())
print(df_sof.shape)

In [ ]:
import p5_util
cluster_start = 1
cluster_end = 10
dict_kmeans = p5_util.kmeans_scan_inter_inertia(df_sof, cluster_start, cluster_end)

**Save Kmeans clustering scan**

In [ ]:
import p5_util

fileName = './data/dict_tfidf_clustering_kmeans.dump'
p5_util.object_dump(dict_kmeans, fileName)

In [ ]:
import p5_util_plot
import p5_util
is_reloaded = False
if is_reloaded is True:
    fileName = './data/dict_tfidf_clustering_kmeans.dump'
    dict_kmeans = p5_util.object_load(fileName)
    print("Records into dict_kmeans= "+str(len(dict_kmeans)))

x_label_0 = 'Features : Nb. clusters'
x_label_1 = 'Features : Nb. clusters'
cluster_start = list(dict_kmeans.keys())[0]
cluster_end = list(dict_kmeans.keys())[len(dict_kmeans)-1]
print(cluster_start, cluster_end)
p5_util_plot.plot_kmeans_interInertia(dict_kmeans, cluster_start, cluster_end\
                             ,[x_label_0, x_label_1]\
                             ,p_rows=1, p_cols=2, p_figsize=(20,10))

In [ ]:
type(dict_kmeans[8])

## 3.3. Word2Vec model

### Embedding corpus with Word2vec model

It is expected that ``word2vec`` learns relationships between words automatically.

* ``df_sof_train['Body']`` represents the corpus with which ``word2vec`` model is feeded.
* Once feed, ``word2vec`` will build a vocabulary, mean, a list of words (features) from which any word from corpus is vectorized.
* Matrix [Corpus words x Features] is stored in ``word2vec_model.wv.vectors``.


In [ ]:
import p6_util
list_sentence = p6_util.get_list_sentence_from_df(df_sof_train, 'Body')

In [ ]:
from gensim.models import Word2Vec
word2vec_model = Word2Vec(list_sentence, min_count=2, size=100, sg=1)

In [ ]:
print(len(word2vec_model.wv.vocab))
len(word2vec_model.wv.vectors)

In [ ]:
word2vec_model.save("word2vec_model")

## Test and performance

In [ ]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
df_sof_test=pd.read_csv(file_name)

### Word2Vec model

* A post is randomly selected from test dataset.
* Then post is ipre-processed in order to feed M.L. model.

In [ ]:
import random
import p6_util

i_random = random.choice(range(0, df_sof_test.shape[0]))
body= df_sof_test.Body.iloc[i_random]
title= df_sof_test.Title.iloc[i_random]
post = body+title
post=title

In [ ]:
post=title
df_post = p6_util.preprocess_post(post)

* Tag counts is computed from test.
* Computed tags are returned with same number and same format then tags from test.

In [ ]:
import p6_util
tag_count=p6_util.get_tag_count(df_sof_test.Tags.iloc[i_random])

list_computed_tag = p6_util.get_list_tag_from_post(df_post.Body.iloc[0], word2vec_model, max_tag=tag_count)
print("Computed tags = "+"".join(list_computed_tag))
print("")
print("Tags from test = "+df_sof_test.Tags.iloc[i_random])
print("------------------------------------------")
print("Posted question : \n")
print("Title : "+str(title))
print("")
print("Body: "+str(body))

## Plot 2D projection of Word2Vec embedded vocabulary

In [ ]:
from gensim.models import Word2Vec

word2vec_model = Word2Vec.load("word2vec_model")

len(word2vec_model.wv.vocab.keys()), len(word2vec_model.wv.vectors)

### Compute matrix for any word into corpus

**Load cleaned train-dataset**

In [ ]:
import p5_util
file_name='./data/df_sof_train.dump'
df_sof_train=p5_util.object_load(file_name)

**Get SOF corpus from dataframe**

In [ ]:
import p6_util
list_corpus = p6_util.get_list_sentence_from_df(df_sof_train, 'Body')

In [ ]:
list_tokenized_corpus=list()
i=0
for list_word in list_corpus:
    list_tokenized_corpus += list_word
    

In [ ]:
len(list_tokenized_corpus[1:]), len(list_tokenized_corpus)

### 2D t-SNE transformation

#### Load TDIF Matrix

In [ ]:
import p5_util
file_name="./data/csr_matrix_tdif.dump"
csr_matrix = p5_util.object_load(file_name)

print(csr_matrix.shape)

**Get sample from CSR matrix**

In [ ]:
import random
import numpy as np
arr = np.zeros(100)
array = csr_matrix.toarray()
#array_index_sample = random.sample(list(csr_matrix.shape), 10)

arr_index = np.random.randint(0,array.shape[0],1000)
array_sample = csr_matrix.toarray()[arr_index]

In [ ]:
array_sample.shape, array.shape

**Compute for embedded vocabulary t-SNE projection with different perplexity values**

In [ ]:
import p5_util
if True:
    #X_std=word2vec_model.wv.vectors
    X_std = array_sample
    my_list_tsne_perplexity=[var for var in range(5,55,5)]
    dict_tsne_result = p5_util.tsne_2D_process_perplexity(X_std, tsne_iter=1000, list_tsne_perplexity=my_list_tsne_perplexity)

**Plot points issued from t-SNE transformations for each perplexity value**

In [ ]:
import p5_util_plot
if True :
    nb_col = 4     
    p5_util_plot.plot_2D_dict_tsne_result(dict_tsne_result, nb_col, ratio=0.05)

**Compute vocabulary t-SNE transformation for a fixed perplexity value**

In [ ]:
import p5_util

X_std=word2vec_model.wv.vectors
print(X_std.shape)
perplexity=10
my_list_tsne_perplexity=[perplexity,]
dict_tsne_result = p5_util.tsne_2D_process_perplexity(X_std, tsne_iter=300, list_tsne_perplexity=my_list_tsne_perplexity)

In [ ]:
len(dict_tsne_result[perplexity]),X_std.shape

**Plot points issued from t-SNE embedded vocabulary transformation with words related to each point**

In [ ]:
import numpy as np
import p5_util_plot

nb_col = 1
dict_tsne_result_part={perplexity:dict_tsne_result[perplexity]}
p_annotation = np.array(list(word2vec_model.wv.vocab.keys()))
p5_util_plot.plot_2D_dict_tsne_result(dict_tsne_result_part, nb_col, ratio=0.05, annotation=p_annotation)

### 2D PCA transformation

In [ ]:
import pandas as pd
df_pca=pd.DataFrame(X_std)

In [ ]:
df_pca.shape

In [ ]:
import p3_util_plot
z__ = p3_util_plot.df_pca_all_plot(df_pca, plot=True)

In [ ]:
from sklearn.decomposition import PCA

nb_components=600
pca = PCA(n_components=nb_components)
pca.fit(X_std)
X_pca = pca.transform(X_std)
print(X_pca.shape)

**Save reduced PCA**

In [ ]:
import p5_util
file_name="./data/pca_600_tdif.dump"
p5_util.object_dump(X_pca, file_name)

In [ ]:
import p3_util_plot
p_ratio=0.1
title = str(int(p_ratio*100))+" % Embedded TF-IDF vocabulary"
pca_=p3_util_plot.df_pca_components_plot(df_pca, None, nb_components=2, param_title=title, ratio=p_ratio)

In [ ]:
df_pca.shape

In [ ]:
import pandas as pd

list(word2vec_model.wv.vocab.keys())
df_vocab=pd.DataFrame(list(word2vec_model.wv.vocab.keys()))

In [ ]:
df_vocab.shape

In [ ]:
import p3_util_plot

p3_util_plot.df_pcs2_plot(df_pca, pca_)